In [1]:
import math
import os
import cv2
import sys
import numpy as np

In [2]:
from skimage.metrics import structural_similarity
import phasepack.phasecong as pc
import cv2

C:\Users\dvmsh\Anaconda3\lib\site-packages\phasepack\tools.py:11: UserWarning: 
Module 'pyfftw' (FFTW Python bindings) could not be imported. To install it, try
running 'pip install pyfftw' from the terminal. Falling back on the slower
'fftpack' module for 2D Fourier transforms.
  warnings.warn("""


In [3]:
def _assert_image_shapes_equal(org_img: np.ndarray, pred_img: np.ndarray, metric: str):
    msg = (f"Cannot calculate {metric}. Input shapes not identical. y_true shape ="
           f"{str(org_img.shape)}, y_pred shape = {str(pred_img.shape)}")

    assert org_img.shape == pred_img.shape, msg

In [4]:
def _gradient_magnitude(img: np.ndarray, img_depth):
    """
    Calculate gradient magnitude based on Scharr operator
    """
    scharrx = cv2.Scharr(img, img_depth, 1, 0)
    scharry = cv2.Scharr(img, img_depth, 0, 1)

    return np.sqrt(scharrx ** 2 + scharry ** 2)

In [5]:
def _similarity_measure(x, y, constant):
    """
    Calculate feature similarity measurement between two images
    """
    numerator = 2 * x * y + constant
    denominator = x ** 2 + y ** 2 + constant

    return numerator / denominator

In [6]:
def fsim(org_img: np.ndarray, pred_img: np.ndarray, T1=0.85, T2=160) -> float:
        _assert_image_shapes_equal(org_img, pred_img, "FSIM")

        alpha = beta = 1  # parameters used to adjust the relative importance of PC and GM features
        fsim_list = []
        for i in range(org_img.shape[2]):
        # Calculate the PC for original and predicted images
            pc1_2dim = pc(org_img[:, :, i], nscale=4, minWaveLength=6, mult=2, sigmaOnf=0.5978)
            pc2_2dim = pc(pred_img[:, :, i], nscale=4, minWaveLength=6, mult=2, sigmaOnf=0.5978)

        # pc1_2dim and pc2_2dim are tuples with the length 7, we only need the 4th element which is the PC.
        # The PC itself is a list with the size of 6 (number of orientation). Therefore, we need to
        # calculate the sum of all these 6 arrays.
            pc1_2dim_sum = np.zeros((org_img.shape[0], org_img.shape[1]), dtype=np.float64)
            pc2_2dim_sum = np.zeros((pred_img.shape[0], pred_img.shape[1]), dtype=np.float64)
            for orientation in range(6):
                pc1_2dim_sum += pc1_2dim[4][orientation]
                pc2_2dim_sum += pc2_2dim[4][orientation]

        # Calculate GM for original and predicted images based on Scharr operator
            gm1 = _gradient_magnitude(org_img[:, :, i], cv2.CV_16U)
            gm2 = _gradient_magnitude(pred_img[:, :, i], cv2.CV_16U)

        # Calculate similarity measure for PC1 and PC2
            S_pc = _similarity_measure(pc1_2dim_sum, pc2_2dim_sum, T1)
        # Calculate similarity measure for GM1 and GM2
            S_g = _similarity_measure(gm1, gm2, T2)

            S_l = (S_pc ** alpha) * (S_g ** beta)

            numerator = np.sum(S_l * np.maximum(pc1_2dim_sum, pc2_2dim_sum))
            denominator = np.sum(np.maximum(pc1_2dim_sum, pc2_2dim_sum))
            fsim_list.append(numerator / denominator)

        return np.mean(fsim_list)

# FSIM for Average Blur

In [30]:
original = cv2.imread(r'C:\Users\dvmsh\Desktop\Study mode\SRGAN Codes\Metrices code\data\HR_images\img10.jpg')
contrast = cv2.imread(r'C:\Users\dvmsh\Desktop\Study mode\SRGAN Codes\Metrices code\data\LR_images\imgs10_avg_blur.jpg',1)

In [31]:
avg = fsim(original , contrast) 
print(avg)

TypeError: 'numpy.float64' object is not callable

# FSIM for Gaussian Blur

In [25]:
original = cv2.imread(r'C:\Users\dvmsh\Desktop\Study mode\SRGAN Codes\Metrices code\data\HR_images\img10.jpg')
contrast = cv2.imread(r'C:\Users\dvmsh\Desktop\Study mode\SRGAN Codes\Metrices code\data\LR_images\imgs10_gaus_blur.jpg',1)

In [29]:
gaus = fsim(original , contrast) 
print(gaus)

TypeError: 'numpy.float64' object is not callable

# FSIM for Median Blur

In [19]:
original = cv2.imread(r'C:\Users\dvmsh\Desktop\Study mode\SRGAN Codes\Metrices code\data\HR_images\img10.jpg')
contrast = cv2.imread(r'C:\Users\dvmsh\Desktop\Study mode\SRGAN Codes\Metrices code\data\LR_images\imgs10_median_blur.jpg',1)

In [20]:
med = fsim(original , contrast) 
print(med)

TypeError: 'numpy.float64' object is not callable

# FSIM for Bilateral Blur

In [21]:
original = cv2.imread(r'C:\Users\dvmsh\Desktop\Study mode\SRGAN Codes\Metrices code\data\HR_images\img10.jpg')
contrast = cv2.imread(r'C:\Users\dvmsh\Desktop\Study mode\SRGAN Codes\Metrices code\data\LR_images\imgs10_bilateral_blur.jpg',1)

In [22]:
fsim = fsim(original , contrast) 
print(fsim)

TypeError: 'numpy.float64' object is not callable